# Data Wrangling with Shapefile Data

<i>The data used in this example: </i>

1) A shapefile created by myself using a fishnet in ArcGIS

2) A Sea Surface Temperature csv fata file gathered from: https://coastwatch.pfeg.noaa.gov/erddap/griddap/jplG1SST.html for 2012 with a bounding box of 32.94N, 35.40N, -120.88W, -117.20W

In [1]:
# Import packages
import pandas as pd # to read in csv file
import datetime as dt # to understand datetime formats
import shapefile # to read in shapefiles
from shapely.geometry import Point # to iterate over features
from shapely.geometry import shape # to iterate over features

In [2]:
# Import data (update to where you stored local files)
grid = shapefile.Reader('D:/Documents/SpringBoard/capstone-1/datasets/trunc/gridded-shapefile') 
    # read in the gridded shapefile
sst = pd.read_csv('D:/Documents/SpringBoard/capstone-1/datasets/trunc/SST_2012.csv', skiprows=1) 
    # read in shapefile and skip the first row (because it's just an extra header)
    # colnames are: UTC, degrees_north, degrees_east, degree_C
    # where degrees_north = latitude and degrees_east = longitude

In [3]:
# Some housekeeping... for SST
sst['UTC'] = pd.to_datetime(sst['UTC'], infer_datetime_format=True, errors='coerce') 
    # change the UTC column to a datetime and coerce non-date formats into NaNs
sst['UTC'] = sst['UTC'].dt.tz_localize('UTC') # give the datetime a tz (utc)
sst = sst[sst['UTC'] == sst['UTC'].min()] # and only keep the sst data for the first day 
    # we really just need to create a dataframe with lat/lon coordinates of SST data paired with the
    # shapefile ID so that we can merge the shapefile ID into the other SST datasets 
sst = sst.dropna().reset_index(drop=True) # only keep the sst data that are not NAs
sst['Zone'] = 'NA' # add a new column that will house the shapefile zone ID

In [4]:
# For the purpose of this example, let's just take the first 9 rows of sst for this day. 
sst = sst.loc[0:9,:]

In [5]:
# Some housekeeping... for the grid
all_shapes = grid.shapes() # get all the polygons
all_records = grid.records() # get all the records

# And just so we get an idea of what we're looking at
print(grid.shapes()[0])
print(grid.records()[0]) # I'm pretty sure this corresponds to [ID, shape , shape area, shape length]

Record #0: [788807, 0, 0.0400000000179, 0.00010000000009]


In [6]:
%%time
# Run the loop
for row in range(len(sst)): # for the rows in sst
    points = (sst['degrees_east'][row], sst['degrees_north'][row]) # put the GPS points from that row 
    # into a tuple
    for shape_id in range(1, len(all_shapes)): # for each of the squares in the shapefile
        boundary = all_shapes[shape_id] # get a boundary polygon
        if Point(points).within(shape(boundary)): # make a point and see if it's in the polygon
            name = all_records[shape_id][0] # grab the field corresponding to the id of the feature
            sst.iloc[row,4] = name # set the feature id as the 5th column (Zone)
            break # and if/when this happens for the GPS tuple, stop the 
            # inner for loop and continue with the outer loop

Wall time: 27.7 s


In [7]:
print(sst) # print out the output

                        UTC  degrees_north  degrees_east  degree_C     Zone
0 2012-01-01 09:00:00+00:00          32.94       -120.88    14.817  1566248
1 2012-01-01 09:00:00+00:00          32.94       -120.87    14.782  1566247
2 2012-01-01 09:00:00+00:00          32.94       -120.86    14.744  1566246
3 2012-01-01 09:00:00+00:00          32.94       -120.85    14.704  1566245
4 2012-01-01 09:00:00+00:00          32.94       -120.84    14.662  1566244
5 2012-01-01 09:00:00+00:00          32.94       -120.83    14.618  1566243
6 2012-01-01 09:00:00+00:00          32.94       -120.82    14.572  1566242
7 2012-01-01 09:00:00+00:00          32.94       -120.81    14.525  1566241
8 2012-01-01 09:00:00+00:00          32.94       -120.80    14.477  1566240
9 2012-01-01 09:00:00+00:00          32.94       -120.79    14.427  1566239


Let's try a new method... this is actually incredibly faster.

In [8]:
import shapely
import geopandas as gdp
import pandas as pd
from geopandas.tools import sjoin
from shapely.geometry import Point
from shapely import speedups
speedups.enable()

In [9]:
# Import data (update to where you stored local files)
grid = gdp.read_file('D:/Documents/SpringBoard/capstone-1/datasets/trunc/gridded-shapefile.shp') 
    # read in the gridded shapefile
sst = pd.read_csv('D:/Documents/SpringBoard/capstone-1/datasets/trunc/SST_2012.csv', skiprows=1) 
    # read in shapefile and skip the first row (because it's just an extra header)
    # colnames are: UTC, degrees_north, degrees_east, degree_C
    # where degrees_north = latitude and degrees_east = longitude

In [10]:
# Some housekeeping... for SST
sst['UTC'] = pd.to_datetime(sst['UTC'], infer_datetime_format=True, errors='coerce') 
    # change the UTC column to a datetime and coerce non-date formats into NaNs
sst['UTC'] = sst['UTC'].dt.tz_localize('UTC') # give the datetime a tz (utc)
sst = sst[sst['UTC'] == sst['UTC'].min()] # and only keep the sst data for the first day 
    # we really just need to create a dataframe with lat/lon coordinates of SST data paired with the
    # shapefile ID so that we can merge the shapefile ID into the other SST datasets 
sst = sst.dropna().reset_index(drop=True) # only keep the sst data that are not NAs

In [11]:
# Turn the SST data into a geometric dataset
geometry = [Point(xy) for xy in zip(sst['degrees_east'], sst['degrees_north'])]

# Coordinate reference system : WGS84
crs = {'init': 'epsg:4326'}

# Create a Geographic data frame 
sst_geom = gdp.GeoDataFrame(sst, crs=crs, geometry=geometry)

# Make sure the grid is the same crs
grid = gdp.GeoDataFrame(grid, crs=crs)
grid = grid.loc[:,['OBJECTID', 'geometry']] # only keep the object ID values and the geometry values

In [12]:
%%time
sst_polygons = sjoin(sst_geom, grid, how='left')

Wall time: 12.5 s


In [13]:
print(sst_polygons.head())

                        UTC  degrees_north  degrees_east  degree_C  \
0 2012-01-01 09:00:00+00:00          32.94       -120.88    14.817   
1 2012-01-01 09:00:00+00:00          32.94       -120.87    14.782   
2 2012-01-01 09:00:00+00:00          32.94       -120.86    14.744   
3 2012-01-01 09:00:00+00:00          32.94       -120.85    14.704   
4 2012-01-01 09:00:00+00:00          32.94       -120.84    14.662   

                      geometry  index_right   OBJECTID  
0  POINT (-120.88000 32.94000)      99622.0  1566248.0  
1  POINT (-120.87000 32.94000)      99621.0  1566247.0  
2  POINT (-120.86000 32.94000)      99620.0  1566246.0  
3  POINT (-120.85000 32.94000)      99619.0  1566245.0  
4  POINT (-120.84000 32.94000)      99618.0  1566244.0  
